In [90]:
import numpy as np

In [91]:
data = open('../data/shakespeare.txt', 'r').read()

In [92]:
data[:100]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with Wo'

In [93]:
chars = list(set(data))

In [94]:
len(chars)

91

In [95]:
data_size, vocab_size = len(data), len(chars)

In [96]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}

In [97]:
char_to_ix

{'}': 0,
 'U': 1,
 '-': 2,
 'x': 3,
 'w': 4,
 'V': 5,
 '4': 6,
 '[': 7,
 'r': 8,
 'g': 9,
 '"': 10,
 'G': 11,
 'W': 12,
 '*': 13,
 '0': 14,
 'K': 15,
 '>': 16,
 'v': 17,
 'H': 18,
 'z': 19,
 '~': 20,
 'e': 21,
 'Z': 22,
 'T': 23,
 '7': 24,
 'Q': 25,
 'E': 26,
 'F': 27,
 'A': 28,
 'I': 29,
 'J': 30,
 't': 31,
 '<': 32,
 'y': 33,
 'd': 34,
 '8': 35,
 '#': 36,
 '|': 37,
 'C': 38,
 'n': 39,
 's': 40,
 'Y': 41,
 ')': 42,
 '3': 43,
 '5': 44,
 'L': 45,
 'M': 46,
 'B': 47,
 ']': 48,
 'c': 49,
 '!': 50,
 'a': 51,
 ' ': 52,
 'P': 53,
 'q': 54,
 '`': 55,
 ';': 56,
 '?': 57,
 'i': 58,
 'S': 59,
 'l': 60,
 'R': 61,
 'b': 62,
 '(': 63,
 'D': 64,
 'j': 65,
 '@': 66,
 'X': 67,
 'h': 68,
 'N': 69,
 '&': 70,
 '2': 71,
 'o': 72,
 '\n': 73,
 'm': 74,
 ':': 75,
 '_': 76,
 '/': 77,
 '6': 78,
 '=': 79,
 'p': 80,
 'k': 81,
 '1': 82,
 '9': 83,
 'f': 84,
 '%': 85,
 "'": 86,
 ',': 87,
 '.': 88,
 'O': 89,
 'u': 90}

In [98]:
ix_to_char = {i:ch for i, ch in enumerate(chars)}

In [99]:
# hyperparameters
hidden_size=100
seq_length = 100
learning_rate = 1e-1

In [100]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(vocab_size, hidden_size)*0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [101]:
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers. 
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0 
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) 
        xs[t][inputs[t]] = 1 # encode in 1-of-k representation
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh,hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) /np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t],0])
    
    #backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1-hs[t]*hs[t]) *dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5,5, out=dparam)
    loss = loss/len(inputs)
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

        
        

In [102]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [103]:
n, p = 0, 0
num_iteration = 10000000000
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)

for ni in range(num_iteration):
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    if n % 100000 == 0:
        sample_ix = sample(hprev, inputs[0],200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('-----\n %s. \n --------' %(txt,))
        
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.9999 + loss * 0.0001 
    if n %10000 == 0: 
        print ('iter %d, loss: %f' % (n, smooth_loss))
    
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam /np.sqrt(mem + 1e-8)
        
    p += seq_length
    n += 1
        
        

-----
 W:huLWzOd;m%I-3X_s]tH=!x?]NB;'L2ix#yckB;hk`oMDf}uFa?oK,Bs&ko!i/%Vi1kRNm`bS[vdDw7*@vPh /eXpq}'*2G` y3"/Yw_SR:bEixFEW;v]:%nhpjQw9('Sl%pgGle8Q02,&X J.6M?;n VeK0<VC':hKp[.Y73EzYQ#vfb>R3iV62[wUg,r*d;jITLF_. 
 --------
iter 0, loss: 4.510860
iter 10000, loss: 2.922935
iter 20000, loss: 2.252044
iter 30000, loss: 1.951117
iter 40000, loss: 1.819196
iter 50000, loss: 1.765986
iter 60000, loss: 1.856310
iter 70000, loss: 1.886993
iter 80000, loss: 1.861640
iter 90000, loss: 1.837951
-----
 at woy,
    And will be knoth your sanst time we trust man me guliover!
  MACTISSIO, be reanaty to hathigO. Is fordy, you croth broveried and ford'ds,







  LONENZABELIZA. O whicher.
  HONTOSSTANO.. 
 --------
iter 100000, loss: 1.802152
iter 110000, loss: 1.791930
iter 120000, loss: 1.765517
iter 130000, loss: 1.766894
iter 140000, loss: 1.744354
iter 150000, loss: 1.725478
iter 160000, loss: 1.724264
iter 170000, loss: 1.711445
iter 180000, loss: 1.719136
iter 190000, loss: 1.697928
-----
        

iter 1610000, loss: 1.540261
iter 1620000, loss: 1.547254
iter 1630000, loss: 1.538676
iter 1640000, loss: 1.543511
iter 1650000, loss: 1.541243
iter 1660000, loss: 1.546562
iter 1670000, loss: 1.540982
iter 1680000, loss: 1.531297
iter 1690000, loss: 1.533613
-----
 ford good gutter lark'd it; hath to like forstand Dorth pas,
    Our given of yoump'rs,
    Sumsitunagess.
  BROAPOMPONEK, was thine, If my braw.
 Exet om noble which of your bounte sir, and, advatce . 
 --------
iter 1700000, loss: 1.533371
iter 1710000, loss: 1.558251
iter 1720000, loss: 1.537210
iter 1730000, loss: 1.540549
iter 1740000, loss: 1.536089
iter 1750000, loss: 1.539611
iter 1760000, loss: 1.544539
iter 1770000, loss: 1.542405
iter 1780000, loss: 1.537606
iter 1790000, loss: 1.531605
-----
 e?
    Gracits, as bagk'd cows?
  VALIOBATBMELAVES. Go,
    The use!
    Than is or porn.
  JALDUTENE SERVANGBUR
                                                                                  Of th. 
 --------
iter 1800

iter 3230000, loss: 1.505855
iter 3240000, loss: 1.531355
iter 3250000, loss: 1.517631
iter 3260000, loss: 1.509447
iter 3270000, loss: 1.516733
iter 3280000, loss: 1.511084
iter 3290000, loss: 1.521583
-----
  restrasour, I care may sports in there tuke heart:
    There will not I Cebelt, stark not advage, qwainerach the least
    To Harerertermows crooking.
  ITUS. The hesmoyery, whonger.
  HENTE. Prawer,. 
 --------
iter 3300000, loss: 1.514772
iter 3310000, loss: 1.516451
iter 3320000, loss: 1.507239
iter 3330000, loss: 1.522314
iter 3340000, loss: 1.507629
iter 3350000, loss: 1.526830
iter 3360000, loss: 1.515282
iter 3370000, loss: 1.506030
iter 3380000, loss: 1.515179
iter 3390000, loss: 1.509778
-----
     Exit WORD to butch, he give to way, owen surthn'd without fray you that so give you dicher's forgir- and mauct
    God and
    cause why, our Tarth rogun, you, no emcies our cares from eye. Alch.
. 
 --------
iter 3400000, loss: 1.522391
iter 3410000, loss: 1.509503
iter 3420

iter 4850000, loss: 1.498203
iter 4860000, loss: 1.508906
iter 4870000, loss: 1.501060
iter 4880000, loss: 1.507702
iter 4890000, loss: 1.501088
-----
 heaven
    Here's turn the betwerity, my    pertwen, quion! Take him, need
    Quest; inter come sure
    Rock an is to scood! 'Tinndly of part secount of
    tenly,
  RISHOL 3. 10
  Ray
    Hath woul. 
 --------
iter 4900000, loss: 1.490011
iter 4910000, loss: 1.496358
iter 4920000, loss: 1.497801
iter 4930000, loss: 1.521582
iter 4940000, loss: 1.498661
iter 4950000, loss: 1.505322
iter 4960000, loss: 1.498562
iter 4970000, loss: 1.505062
iter 4980000, loss: 1.507070
iter 4990000, loss: 1.505138
-----
 d vour than a freth roubs, 
    This bene, Veen-let whole, swisements my of the means
    That conofgealkins, if I am. Come; and morry I wascals? You should  lip all
    I sweeck
    And Cat, but grea. 
 --------
iter 5000000, loss: 1.499133
iter 5010000, loss: 1.494185
iter 5020000, loss: 1.493318
iter 5030000, loss: 1.492329
iter 5040

iter 6470000, loss: 1.500414
iter 6480000, loss: 1.490824
iter 6490000, loss: 1.497390
-----
  break eyes.
    What.
  POSSY.     Exit
  PERD Pho dooes sndeas, what.
    Ore.
  GRATEUS
        Exeunindom and eals
    Look answeres jur lisdandars a plack so caigh requye life!"
  SANDAR BAT the . 
 --------
iter 6500000, loss: 1.493986
iter 6510000, loss: 1.504775
iter 6520000, loss: 1.497826
iter 6530000, loss: 1.497075
iter 6540000, loss: 1.490128
iter 6550000, loss: 1.504333
iter 6560000, loss: 1.489557
iter 6570000, loss: 1.511454
iter 6580000, loss: 1.498621
iter 6590000, loss: 1.489604
-----
 nd.
  ACHINIUS having man.
  CLOWN. Come
    Master. Is plach hearfy dost-viage, his passed Were, thyself affectiol, a soul hindevor too three conjution;
    To untry no poer'd.
  GUATED. CUTRITANGETE. 
 --------
iter 6600000, loss: 1.497132
iter 6610000, loss: 1.491956
iter 6620000, loss: 1.505973
iter 6630000, loss: 1.493895
iter 6640000, loss: 1.499708
iter 6650000, loss: 1.490841
iter 6660

...

iter 64110000, loss: 1.474279
iter 64120000, loss: 1.460162
iter 64130000, loss: 1.474276
iter 64140000, loss: 1.481132
iter 64150000, loss: 1.499855
iter 64160000, loss: 1.475219
iter 64170000, loss: 1.480434
iter 64180000, loss: 1.471580
iter 64190000, loss: 1.485642
-----
 r, mindience. Fay,
    I graves of ip you do of the stand, Cogch.  
    Open in the
    kneck; capest togethyA
  But. How into thine sir.
  Englots,
     
    Houseprs that; fuls, and will to the bani. 
 --------
iter 64200000, loss: 1.483029
iter 64210000, loss: 1.481142
iter 64220000, loss: 1.476073
iter 64230000, loss: 1.466670
iter 64240000, loss: 1.467998
iter 64250000, loss: 1.474152
iter 64260000, loss: 1.497925
iter 64270000, loss: 1.475579
iter 64280000, loss: 1.475514
iter 64290000, loss: 1.471416
-----
 n the heart teething?           Enter but then,
    Stain, of wise not it good Gon a are forthin's man; borney Had that the mins I have I
    one withordle. Go with the we cheeren, I will must in the . 


iter 65710000, loss: 1.476035
iter 65720000, loss: 1.478411
iter 65730000, loss: 1.488911
iter 65740000, loss: 1.479587
iter 65750000, loss: 1.475526
iter 65760000, loss: 1.470653
iter 65770000, loss: 1.468151
iter 65780000, loss: 1.471881
iter 65790000, loss: 1.494945
-----
   Of test imardmine in which about consing love majused, for to Roman.    [The will as Master were at the preds?
    (butireds. Yest lov'ds be lise you stand,
    Now, good make to Wither,
    Thou sh. 
 --------
iter 65800000, loss: 1.480376
iter 65810000, loss: 1.470322
iter 65820000, loss: 1.476302
iter 65830000, loss: 1.476671
iter 65840000, loss: 1.487531
iter 65850000, loss: 1.477705
iter 65860000, loss: 1.480383
iter 65870000, loss: 1.470591
iter 65880000, loss: 1.489321
iter 65890000, loss: 1.475469
-----
 herefore the son fall.
    He
    and none,
    Fire? Look with the'el-kina be far suck, fall in Duke's one murdar shake on
    Yourse!
  KING RERFLON. The emity'd no a fair thou boys
    To stropouce. 


iter 67310000, loss: 1.477892
iter 67320000, loss: 1.483485
iter 67330000, loss: 1.475528
iter 67340000, loss: 1.463087
iter 67350000, loss: 1.475119
iter 67360000, loss: 1.480128
iter 67370000, loss: 1.497959
iter 67380000, loss: 1.475906
iter 67390000, loss: 1.482054
-----
 he less you side undeeht,
    So in peeticy.
  PISTOL. That unto then, you catufalter name weilling picrive!
  SECISSAY, where sain at heavencohard. And had for stands your loart
    Go Well stopp 
  . 
 --------
iter 67400000, loss: 1.471938
iter 67410000, loss: 1.486929
iter 67420000, loss: 1.481067
iter 67430000, loss: 1.483029
iter 67440000, loss: 1.477008
iter 67450000, loss: 1.466154
iter 67460000, loss: 1.470823
iter 67470000, loss: 1.476369
iter 67480000, loss: 1.498643
iter 67490000, loss: 1.475147
-----
 e grout you, we from his greates! If live.
    When yuch in by where with here sleepy love, her!

    If the give
    I first stephord a stand lose?
  KING. Why bepong thou all.

                    [. 


iter 68910000, loss: 1.481241
iter 68920000, loss: 1.470554
iter 68930000, loss: 1.473945
iter 68940000, loss: 1.481597
iter 68950000, loss: 1.488489
iter 68960000, loss: 1.481247
iter 68970000, loss: 1.476022
iter 68980000, loss: 1.471078
iter 68990000, loss: 1.469300
-----
 m bow her thy be another, Caisors.
  FIRST WIT SOENCLOR. Conise. Let my
    fall by prides; in see ques's for set I kipus; whol's drink such with our sour my liotes, face?
  YORK. D letsure it say?   . 
 --------
iter 69000000, loss: 1.472794
iter 69010000, loss: 1.494332
iter 69020000, loss: 1.482920
iter 69030000, loss: 1.470840
iter 69040000, loss: 1.477218
iter 69050000, loss: 1.478080
iter 69060000, loss: 1.489150
iter 69070000, loss: 1.479311
iter 69080000, loss: 1.481141
iter 69090000, loss: 1.469693
-----
 owlmissa;
  Beath sir!
  Vee.
  Prince;
    To lattes levenbeid
    By that
  Foou soldiers,
    By Veef a sleep peingets renting,
  Pornes to APESLOR. [Reads
    They be sorn not dogare this man. Tho. 


KeyboardInterrupt: 